In [ ]:
import sys

sys.path.append("./ImageBind/")

Пробуем добавить очки собакам

In [ ]:
import imagebind
import torch
from diffusers import StableUnCLIPImg2ImgPipeline

device = "cuda:0" if torch.cuda.is_available() else "cpu"
pipe = StableUnCLIPImg2ImgPipeline.from_pretrained(
    "stabilityai/stable-diffusion-2-1-unclip", torch_dtype=torch.float32
)
pipe = pipe.to(device)

model = imagebind.imagebind_model.imagebind_huge(pretrained=True)
model.eval()
model.to(device)

for img_w in [0.2, 0.4, 0.5, 0.6, 0.8]:
    with torch.no_grad():
        paths = ["assets/orig_scottish.png"]
        embeddings = model.forward(
            {
                imagebind.imagebind_model.ModalityType.VISION: imagebind.data.load_and_transform_vision_data(
                    paths, device
                ),
                imagebind.imagebind_model.ModalityType.TEXT: imagebind.data.load_and_transform_text(
                    ["sunglasses"], device
                ),
            }
        )
        embeddings = (
            img_w * embeddings[imagebind.imagebind_model.ModalityType.VISION]
            + (1 - img_w) * embeddings[imagebind.imagebind_model.ModalityType.TEXT]
        ).view(1, -1)
        images = pipe(image_embeds=embeddings, num_inference_steps=30).images
        images[0].save(f"experiments/dogsunglasses_v2_orig{img_w}.png")

Text+Text -> Image

In [ ]:
for img_w in [0.2, 0.4, 0.5, 0.6, 0.8]:
    with torch.no_grad():
        paths = ["assets/orig_scottish.png"]
        embeddings = model.forward(
            {
                imagebind.imagebind_model.ModalityType.TEXT: imagebind.data.load_and_transform_text(
                    ["dog", "sunset"], device
                ),
            }
        )
        embeddings = (
            img_w * embeddings[imagebind.imagebind_model.ModalityType.TEXT][0]
            + (1 - img_w) * embeddings[imagebind.imagebind_model.ModalityType.TEXT][1]
        ).view(1, -1)
        images = pipe(image_embeds=embeddings, num_inference_steps=30).images
        images[0].save(f"experiments/texttext_v2_dog{img_w}.png")

Text -> Image

Image -> Image

Audio -> Image

In [ ]:
with torch.no_grad():
    paths = ["assets/orig_scottish.png"]
    embeddings = model.forward(
        {
            imagebind.imagebind_model.ModalityType.TEXT: imagebind.data.load_and_transform_text(
                ["sunset"], device
            ),
        }
    )
    embeddings = (
        embeddings[imagebind.imagebind_model.ModalityType.TEXT]
    ).view(1, -1)
    images = pipe(image_embeds=embeddings, num_inference_steps=30).images
    images[0].save(f"experiments/text2img.png")

with torch.no_grad():
    paths = ["assets/orig_scottish.png"]
    embeddings = model.forward(
        {
            imagebind.imagebind_model.ModalityType.VISION: imagebind.data.load_and_transform_vision_data(
                paths, device
            ),
        }
    )
    embeddings = (
        embeddings[imagebind.imagebind_model.ModalityType.VISION]
    ).view(1, -1)
    images = pipe(image_embeds=embeddings, num_inference_steps=30).images
    images[0].save(f"experiments/img2img.png")

with torch.no_grad():
    paths = ["assets/wave.wav"]
    embeddings = model.forward(
        {
            imagebind.imagebind_model.ModalityType.AUDIO: imagebind.data.load_and_transform_audio_data(
                paths, device
            ),
        }
    )
    embeddings = (
        embeddings[imagebind.imagebind_model.ModalityType.AUDIO]
    ).view(1, -1)
    images = pipe(image_embeds=embeddings, num_inference_steps=30).images
    images[0].save(f"experiments/audio2img.png")

Image+Image -> Image

In [ ]:
for img_w in [0.2, 0.4, 0.6, 0.8]:
    with torch.no_grad():
        paths = ["assets/orig_scottish.png", "assets/Drake_-_For_All_The_Dogs.png"]
        embeddings = model.forward(
            {
                imagebind.imagebind_model.ModalityType.VISION: imagebind.data.load_and_transform_vision_data(
                    paths, device
                )
            }
        )
        embeddings = (
            img_w * embeddings[imagebind.imagebind_model.ModalityType.VISION][0]
            + (1 - img_w) * embeddings[imagebind.imagebind_model.ModalityType.VISION][1]
        ).view(1, -1)
        images = pipe(image_embeds=embeddings, num_inference_steps=30).images
        images[0].save(f"experiments/imgimg2img_v2_orig{img_w}.png")

Image+Text -> Image

In [ ]:
for img_w in [0.2, 0.4, 0.6, 0.8]:
    with torch.no_grad():
        paths = ["assets/orig_scottish.png"]
        embeddings = model.forward(
            {
                imagebind.imagebind_model.ModalityType.VISION: imagebind.data.load_and_transform_vision_data(
                    paths, device
                ),
                imagebind.imagebind_model.ModalityType.TEXT: imagebind.data.load_and_transform_text(
                    ["sunset"], device
                ),
            }
        )
        embeddings = (
            img_w * embeddings[imagebind.imagebind_model.ModalityType.VISION]
            + (1 - img_w) * embeddings[imagebind.imagebind_model.ModalityType.TEXT]
        ).view(1, -1)
        images = pipe(image_embeds=embeddings, num_inference_steps=30).images
        images[0].save(f"experiments/imgtext2img_v2_orig{img_w}.png")

Try to remove dog by subtracting "dog" vector from image vector

In [ ]:
for img_w in [0.5, 0.75, 1.0, 1.25, 1.5]:
    with torch.no_grad():
        paths = ["assets/orig_scottish.png"]
        embeddings = model.forward(
            {
                imagebind.imagebind_model.ModalityType.VISION: imagebind.data.load_and_transform_vision_data(
                    paths, device
                ),
                imagebind.imagebind_model.ModalityType.TEXT: imagebind.data.load_and_transform_text(
                    ["dog"], device
                ),
            }
        )
        embeddings = (
            embeddings[imagebind.imagebind_model.ModalityType.VISION]
            - img_w * embeddings[imagebind.imagebind_model.ModalityType.TEXT]
        ).view(1, -1)
        images = pipe(image_embeds=embeddings, num_inference_steps=30).images
        images[0].save(f"experiments/substr_imgtext2img_v2_text{img_w}.png")

Image+Audio+Text -> Image

In [ ]:
for img_w in [0.25, 0.5, 0.75]:
    for text_w in [0.25, 0.5, 0.75]:
        for audio_w in [0.25, 0.5, 0.75]:
            with torch.no_grad():
                paths = ["assets/orig_scottish.png"]
                audio_paths = ["assets/wave.wav"]
                embeddings = model.forward(
                    {
                        imagebind.imagebind_model.ModalityType.VISION: imagebind.data.load_and_transform_vision_data(
                            paths, device
                        ),
                        imagebind.imagebind_model.ModalityType.TEXT: imagebind.data.load_and_transform_text(
                            ["sunset"], device
                        ),
                        imagebind.imagebind_model.ModalityType.AUDIO: imagebind.data.load_and_transform_audio_data(
                            audio_paths, device
                        ),
                    }
                )
                embeddings = (
                    img_w * embeddings[imagebind.imagebind_model.ModalityType.VISION]
                    + text_w * embeddings[imagebind.imagebind_model.ModalityType.TEXT]
                    + audio_w * embeddings[imagebind.imagebind_model.ModalityType.AUDIO]
                )
                images = pipe(image_embeds=embeddings, num_inference_steps=30).images
                images[0].save(f"experiments/audiotextimg_v2_{img_w}_{text_w}_{audio_w}.png")

Text+Audio -> Image

In [ ]:
for img_w in [0.2, 0.4, 0.5, 0.6, 0.8]:
    with torch.no_grad():
        paths = ["assets/wave.wav"]
        embeddings = model.forward(
            {
                imagebind.imagebind_model.ModalityType.AUDIO: imagebind.data.load_and_transform_audio_data(
                    paths, device
                ),
                imagebind.imagebind_model.ModalityType.TEXT: imagebind.data.load_and_transform_text(
                    ["sunset"], device
                ),
            }
        )
        embeddings = (
            img_w * embeddings[imagebind.imagebind_model.ModalityType.AUDIO]
            + (1 - img_w) * embeddings[imagebind.imagebind_model.ModalityType.TEXT]
        ).view(1, -1)
        images = pipe(image_embeds=embeddings, num_inference_steps=30).images
        images[0].save(f"experiments/audiotext2img_v2_audio{img_w}.png")

Audio+Image -> Image

In [ ]:
for img_w in [0.2, 0.4, 0.5, 0.6, 0.8]:
    with torch.no_grad():
        audio_paths = ["assets/wave.wav"]
        img_paths = ["assets/orig_scottish.png"]
        embeddings = model.forward(
            {
                imagebind.imagebind_model.ModalityType.AUDIO: imagebind.data.load_and_transform_audio_data(
                    audio_paths, device
                ),
                imagebind.imagebind_model.ModalityType.VISION: imagebind.data.load_and_transform_vision_data(
                    img_paths, device
                ),
            }
        )
        embeddings = (
            img_w * embeddings[imagebind.imagebind_model.ModalityType.AUDIO]
            + (1 - img_w) * embeddings[imagebind.imagebind_model.ModalityType.VISION]
        ).view(1, -1)
        images = pipe(image_embeds=embeddings, num_inference_steps=30).images
        images[0].save(f"experiments/audioimg2img_v2_audio{img_w}.png")

Trying to remove sunglasses from the dog

In [ ]:
for img_w in [0.2, 0.4, 0.5, 0.6, 0.8]:
    with torch.no_grad():
        paths = ["assets/dogsunglasses_v2_orig0.6.png"]
        embeddings = model.forward(
            {
                imagebind.imagebind_model.ModalityType.VISION: imagebind.data.load_and_transform_vision_data(
                    paths, device
                ),
                imagebind.imagebind_model.ModalityType.TEXT: imagebind.data.load_and_transform_text(
                    ["sunglasses"], device
                ),
            }
        )
        embeddings = (
            embeddings[imagebind.imagebind_model.ModalityType.VISION]
            - (1 - img_w) * embeddings[imagebind.imagebind_model.ModalityType.TEXT]
        ) / img_w
        images = pipe(image_embeds=embeddings, num_inference_steps=30).images
        images[0].save(f"experiments/substr_sunglasses2img_v2_text{img_w}.png")